# GPT-4 Fine-tuning으로 Summarize 하기

# STT 결과를 자동으로 합치는 Python 코드 : 여러 개의 STT 결과 파일(local폴더에 있는)을 하나의 프롬프트에 넣을 수 있도록

In [36]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# 🔹 .env 파일 로드
load_dotenv()

# 🔹 환경 변수에서 Azure OpenAI 설정 가져오기
api_key = os.getenv("AZURE_OPENAI_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
api_version="2024-08-01-preview"

# 🔹 Azure OpenAI 클라이언트 생성 (수정된 부분)
client = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=endpoint,
    azure_deployment=deployment_name,
    api_version=api_version
)

# 🔹 STT 결과가 저장된 폴더 경로 (STT 결과 파일이 여러 개 있다고 가정)
stt_folder = "STTresults/1"  # STT 결과 파일이 저장된 폴더 경로

# 🔹 폴더 내 모든 STT 텍스트 파일을 불러와서 합치기
stt_texts = []
for filename in sorted(os.listdir(stt_folder)):  # 파일명 순서대로 정렬
    if filename.endswith(".txt"):  # STT 결과 파일이 .txt라고 가정
        with open(os.path.join(stt_folder, filename), "r", encoding="utf-8") as file:
            stt_texts.append(file.read().strip())

# 🔹 모든 STT 결과를 하나로 합침
full_stt_text = "\n".join(stt_texts)

# 🔹 프롬프트 생성
prompt = f"""
당신은 환자와 의사의 대화를 분석하는 AI입니다.  
아래의 텍스트에서 의학적으로 중요한 정보만 요약하세요.  
진단, 치료 계획, 부작용, 수술 내용, 수술 과정, 주의사항 등 **필수적인 의료 정보만 포함**해야 합니다.  
불필요한 잡담이나 감정적인 대화는 **모두 제거**하십시오.  
요약은 간결하고 정확하게 작성하세요.
이유를 설명하는 내용은 빼지마세요.

입력 데이터:
\"\"\" {full_stt_text} \"\"\"

출력 형식:
- **진단:** [질병명 또는 상태]
- **수술/치료 계획:** [필요한 절차]
- **부작용 및 위험:** [주요 부작용]
- **주의사항:** [환자가 주의해야 할 사항]
- **기타:** [부가적인 설명]
"""

# 🔹 Azure OpenAI API 호출 (수정된 부분)
response = client.chat.completions.create(
    model=deployment_name, 
    messages=[{"role": "system", "content": "You are an AI assistant specialized in summarizing medical conversations."},
              {"role": "user", "content": prompt}]
)

# 🔹 결과 출력
print(response.choices[0].message.content)


- **진단:** 급성 담낭염
- **수술/치료 계획:** 복강경 담낭절제술 예정, 필요 시 개복수술로 전환 가능
- **부작용 및 위험:** 수술 중 출혈, 담즙 누출, 담관 손상, 감염 및 복강 내 농양, 전신마취 합병증 가능성
- **주의사항:** 
  - 수술 후 통증은 진통제로 조절
  - 수술 후 1~2주간 무리한 신체활동 자제
  - 상처 부위 청결 유지, 이상 시 병원 방문
  - 소화가 잘 되는 음식 섭취 권장
  - 당뇨 및 고혈압 약물 꾸준히 복용
- **기타:** 
  - 복강경 수술은 회복이 빠르고 흉터가 작음
  - 기름진 음식을 섭취 시 초기 소화 불편 가능하지만 적응 가능
  - 혈압과 혈당 관리 중요, 수술 후에도 정상적인 생활 가능


In [41]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# 🔹 .env 파일 로드
load_dotenv()

# 🔹 환경 변수에서 Azure OpenAI 설정 가져오기
api_key = os.getenv("AZURE_OPENAI_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
api_version = "2024-08-01-preview"

# 🔹 Azure OpenAI 클라이언트 생성 (수정된 부분)
client = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=endpoint,
    azure_deployment=deployment_name,
    api_version=api_version
)

# 🔹 STT 결과가 저장된 폴더 경로 (STT 결과 파일이 여러 개 있다고 가정)
stt_folder = "STTresults/1"  # STT 결과 파일이 저장된 폴더 경로

# 🔹 폴더 내 모든 STT 텍스트 파일을 불러와서 합치기
stt_texts = []
for filename in sorted(os.listdir(stt_folder)):  # 파일명 순서대로 정렬
    if filename.endswith(".txt"):  # STT 결과 파일이 .txt라고 가정
        with open(os.path.join(stt_folder, filename), "r", encoding="utf-8") as file:
            stt_texts.append(file.read().strip())

# 🔹 모든 STT 결과를 하나로 합침
full_stt_text = "\n".join(stt_texts)

# 🔹 합친 텍스트를 같은 폴더에 저장
merged_file_path = os.path.join(stt_folder, "merged_stt.txt")
with open(merged_file_path, "w", encoding="utf-8") as merged_file:
    merged_file.write(full_stt_text)

print(f"📂 STT 결과가 합쳐진 파일이 저장되었습니다: {merged_file_path}")

# 🔹 프롬프트 생성
prompt = f"""
당신은 환자와 의사의 대화를 분석하는 AI입니다.  
아래의 대화 내용을 기반으로 **핵심 정보는 유지하면서도 짧고 간결한 요약**을 생성하세요.  

❗ 단, 중요한 내용이 삭제되지 않도록 하며, 설명이 필요하면 간결한 문장으로 추가하세요.  
❗ 정보가 너무 축약되지 않도록 하며, **한 항목당 2~4개의 핵심 내용을 유지하세요.**  
❗ 줄글이 아닌, `진단:`, `수술/치료 계획:` 등의 키워드 형식을 유지하세요.  
❗ **부작용 및 위험** 항목에서는 주요 합병증(담관 손상, 담즙 누출, 출혈 등)에 대해 개별적으로 설명하세요. 이 부분만 줄글이고 나머지는 키워드 형식 유지.

입력 데이터:
\"\"\" {full_stt_text} \"\"\"  

### **출력 형식 (각 항목을 간결하지만 핵심 내용을 유지하면서 요약)**  
- **진단:** [질병명]  
  - 예: 급성 담낭염  

- **수술/치료 계획:** [필요한 절차, 치료 선택 이유, 치료 효과, 수술과 관련된 주요 문제점]  
  - 예: 복강경 담낭절제술
  - 예: 담낭염이 생기거나 용종 등 이상이 있으면 담낭을 제거함
  - 예: 수술 방법
  - 예: 이 방법 선택 이유

- **부작용 및 위험:** [예상 가능한 부작용, 합병증 발생 가능성, 합병증 발생 시 대처법]  
  - **담관 손상:** 담낭을 제거하는 과정에서 담관이 손상될 수 있습니다. 담관이 손상되면 담즙의 정상적인 흐름이 방해받아 황달이나 소화 장애가 발생할 수 있습니다. 심한 경우 추가적인 수술이나 스텐트 삽입이 필요할 수 있습니다.  
  - **담즙 누출:** 담낭 제거 후 담즙이 새어나올 가능성이 있으며, 이는 복강 내 감염이나 염증을 유발할 수 있습니다. 치료 방법으로는 배액관을 삽입하거나 추가적인 수술이 필요할 수 있습니다.  
  - **출혈:** 수술 중 혈관이 손상되거나 혈액 응고 장애가 있는 경우 출혈이 발생할 수 있습니다. 출혈이 심할 경우 수혈이 필요할 수 있으며, 지혈이 어려울 경우 개복 수술로 전환될 가능성이 있습니다.  

- **주의사항:** [수술 전후 환자가 지켜야 할 사항, 생활습관 조정, 주의해야 할 합병증]  
  - 예: 수술 후 출혈 (고혈압 환자의 출혈 위험 증가) , 유의해야할 warning signs

- **기타:** [환자의 질문과 답변, 기타 중요한 설명]  
  - 예: 수술 후 식습관 조정 (지방 흡수 장애)  , 통증 조절
"""

# 🔹 Azure OpenAI API 호출 (수정된 부분)
response = client.chat.completions.create(
    model=deployment_name, 
    messages=[{"role": "system", "content": "You are an AI assistant specialized in summarizing medical conversations."},
              {"role": "user", "content": prompt}]
)

# 🔹 결과 출력
print(response.choices[0].message.content)


📂 STT 결과가 합쳐진 파일이 저장되었습니다: STTresults/1\merged_stt.txt
- **진단:** 급성 담낭염  
  - 담낭에 급성으로 염증이 발생하였음.

- **수술/치료 계획:** 복강경 담낭절제술  
  - 담낭염 또는 용종 등 이상 시 담낭을 제거함.
  - 수술은 전신마취하에 진행되며 복부에 3~4개의 작은 절개를 내어 진행.
  - 개복수술보다 회복이 빠르고 합병증 발생 위험이 적어 이 방법을 선택.

- **부작용 및 위험:**
  - **담관 손상:** 담낭을 제거하는 과정에서 손상 가능. 담즙 흐름 방해, 황달, 소화 장애 발생 가능. 경우에 따라 추가 수술 필요.
  - **담즙 누출:** 제거 후 담즙 누출로 복강 내 감염이나 염증 유발 가능. 배액관 삽입 또는 추가 수술 필요할 수 있음.
  - **출혈:** 혈관 손상 또는 혈액 응고 장애로 인한 출혈 가능성. 필요시 수혈, 지혈 어려울 경우 개복 수술 전환 가능.

- **주의사항:** 
  - 수술 후 무리한 신체활동은 피하고, 상처 부위는 청결하게 유지.
  - 상처 부위가 붓거나 발열, 고름이 생길 시 즉시 병원 방문.
  - 고혈압과 당뇨와 같은 기저질환 관리가 중요. 수술 전후 혈압 및 혈당 조절 필수.

- **기타:** 
  - 수술 후 지방 섭취 시 소화 불편 가능성. 식사는 기름기 없는 식품 위주로 조절.
  - 수술 후 통증은 진통제로 관리 가능하나, 통증이 더욱 심해지면 병원 방문.
  - 수술 이후 일반적인 활동은 회복 후 1~2주 내 가능하나, 육체적으로 무리한 활동은 한 달가량 피함.
